In [1]:
from aeon.datasets import load_classification
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import time

from ConvBN1d import ConvBN
from LinearBN import LinearBN

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_name = 'WalkingSittingStanding'
# Load dataset
X_train, y_train, metadata = load_classification(dataset_name, return_metadata=True, split='train')
X_test, y_test = load_classification(dataset_name, split='test')
if X_train.shape[0] < 200:
    train_size = int((X_train.shape[0] + X_test.shape[0]) * 3/4)
    x, y = load_classification(dataset_name)
    X_train, y_train = x[:train_size, :], y[:train_size]
    X_test, y_test = x[train_size:, :], y[train_size:]

print(f'Train Size Used: {X_train.shape[0]}, Test Size Used: {X_test.shape[0]}')

# Flatten X if shape is (n_samples, 1, series_length) → (n_samples, series_length)
input_channels = 1
if X_train.ndim == 3:
    input_channels = X_train.shape[1]
    X_train = np.squeeze(X_train, axis=1) if input_channels == 1 else X_train
    X_test = np.squeeze(X_test, axis=1) if input_channels == 1 else X_test

seq_length = X_train.shape[-1]  # series length

# Encode labels
if y_train.dtype == object or isinstance(y_train[0], str):
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)

# Standard scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, seq_length))
X_test_scaled = scaler.transform(X_test.reshape(-1, seq_length))

Train Size Used: 7352, Test Size Used: 2947


In [3]:
# ---- CNN ----
if input_channels == 1:
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).unsqueeze(1).to(device)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).unsqueeze(1).to(device)
else:
    # Multichannel → keep original channels
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

num_classes = len(np.unique(y_train))

print(X_train.shape)
print(X_test.shape)

(7352, 3, 206)
(2947, 3, 206)


In [4]:
class LReLU(nn.Module):
    def __init__(self):
        super(LReLU, self).__init__()
        self.alpha = nn.Parameter(torch.tensor(5.0)) 
    def forward(self, x):
        return torch.nn.functional.relu(self.alpha*x)

In [5]:
class Network(nn.Module):
    def __init__(self, inpu_channels, seq_length, num_classes):
        super(Network, self).__init__()

        self.conv1_out = 64
        self.conv1_size = 11
        self.conv1_padding = 5


        self.conv2_out = 48
        self.conv2_size = 7
        self.conv2_padding = 3

        self.conv3_out = 32
        self.conv3_size = 3
        self.conv3_padding = 1

        self.fc1_out = num_classes

        self.q = 1e-6
        self.bias_trick_par = nn.Parameter(torch.tensor(0.00005))

        # First Convolutional Block
        # Seed 1 and 2

        self.block1 = ConvBN(in_channels=input_channels, out_channels=self.conv1_out, kernel_size=self.conv1_size, padding=self.conv1_padding, std = .01, bias_par_init=0.0015)
        self.block2 = ConvBN(in_channels=self.conv1_out, out_channels=self.conv2_out, kernel_size=self.conv2_size, padding=self.conv2_padding, std = .01, bias_par_init=0.0015)
               
        
        torch.manual_seed(0)
        self.w2 = nn.Parameter(torch.randn(self.conv2_out * (seq_length// 2 // 2), self.fc1_out))
        nn.init.normal_(self.w2, mean=0.0, std=.1)

        self.dropout = nn.Dropout(0.5)

        self.relu = LReLU()

        self.pool = nn.MaxPool1d(2)




    def forward(self, x):
        x = self.pool(self.relu(self.block1(x)))
        x = self.pool(self.relu(self.block2(x)))
        
        x = x.view(x.size(0), -1)
        
        x = x + self.bias_trick_par
        x_norm = x / (x.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize input x
        w2_norm = self.w2 / (self.w2.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize weights
        x = torch.matmul(x_norm, w2_norm) # Matrix multiplication 

        # Return raw logits (no softmax here, CrossEntropyLoss handles it)
        return x

In [6]:
num_runs = 1
test_accuracies = []

for run in range(1, num_runs + 1):
    model = Network(input_channels, seq_length, num_classes).to(device)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.00001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

    # ----- TRAIN -----
    model.train()
    for epoch in range(120):
        start = time.time()
        running_loss = 0.0
        correct = 0
        for batch, labels in train_loader:
            batch, labels = batch.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(batch)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * batch.size(0)
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()

        avg_loss = running_loss / len(train_dataset)
        avg_acc  = correct     / len(train_dataset)
        scheduler.step(avg_loss)  # adjust LR if plateau

        elapsed = time.time() - start
        if (epoch+1) % 20 == 0:
            print(f"[Run {run}/{num_runs}] Epoch {epoch+1:02d} | "
              f"loss: {avg_loss:.4f} | acc: {avg_acc:.4f} | time: {elapsed:.1f}s")

    # ----- TEST -----
    model.eval()
    test_loss = 0.0
    correct_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            test_loss += criterion(outputs, labels).item() * inputs.size(0)
            _, preds = outputs.max(1)
            correct_test += (preds == labels).sum().item()

    avg_test_loss = test_loss / len(test_dataset)
    avg_test_acc  = correct_test / len(test_dataset)
    test_accuracies.append(avg_test_acc)

    print(f"→ Run {run} Test loss: {avg_test_loss:.4f}, Test acc: {avg_test_acc:.4f}")

# ----- SUMMARY -----
mean_acc = np.mean(test_accuracies)
std_acc  = np.std(test_accuracies)
print(f"Average test accuracy over {num_runs} runs: {mean_acc:.4f} ± {std_acc:.4f}")

[Run 1/1] Epoch 20 | loss: 0.1078 | acc: 0.9559 | time: 0.2s
[Run 1/1] Epoch 40 | loss: 0.1011 | acc: 0.9611 | time: 0.2s
[Run 1/1] Epoch 60 | loss: 0.0912 | acc: 0.9637 | time: 0.2s
[Run 1/1] Epoch 80 | loss: 0.0909 | acc: 0.9638 | time: 0.2s
[Run 1/1] Epoch 100 | loss: 0.0909 | acc: 0.9638 | time: 0.2s
[Run 1/1] Epoch 120 | loss: 0.0909 | acc: 0.9638 | time: 0.2s
→ Run 1 Test loss: 0.5849, Test acc: 0.8561
Average test accuracy over 1 runs: 0.8561 ± 0.0000


In [7]:
model.eval()  # Set the model to evaluation mode

test_loss = 0.0
correct_test = 0

# Evaluate on the test dataset
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Get predictions and update the correct count
        _, predicted = torch.max(outputs, 1)
        correct_test += (predicted == labels).sum().item()

# Compute average loss and accuracy for the test set
avg_test_loss = test_loss / len(test_dataset)
avg_test_acc = correct_test / len(test_dataset)

print(f"Test loss: {avg_test_loss:.5f}, Test accuracy: {avg_test_acc:.4f}")

Test loss: 0.00115, Test accuracy: 0.8561


In [8]:
torch.save(model.state_dict(), f'{dataset_name}_GNet_Training_{avg_test_acc:.4f}.pth')

In [9]:
print(f'{avg_test_acc:.4f}')

0.8561


In [10]:
torch.save(avg_test_acc, 'avg_test_acc.pt')